# MNIST classification with kNN classifier

# Gladyshev Alexey

## 1. Load MNIST dataset and prepare data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from copy import deepcopy
from tqdm import tqdm

In [2]:
data_path_prefix = '../Datasets/MNIST'

In [3]:
train_data = pd.read_csv(os.path.join(data_path_prefix, 'mnist_train.csv'))
test_data  = pd.read_csv(os.path.join(data_path_prefix, 'mnist_test.csv'))

In [4]:
test_data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_train = train_data.drop(('label'), axis = 1)
y_train = train_data['label']
print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [6]:
X_test = test_data.drop(('label'), axis = 1)
y_test = test_data['label']
print(X_test.shape)
print(y_test.shape)

(10000, 784)
(10000,)


In [7]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

## 2. kNN classifier implementation

In [8]:
class KNeighborsClassifier:
    
    def __init__(self, distance_func, k=5):
        self.k = k
        self.distance_func = distance_func
        
    def fit(self, X, y):
        self.__X = X
        self.__y = y
        
        
    def predict_proba(self, X):
        result = []
        nearest_neighbors_labels = self.__get_nearest_neighbors_labels(X)
        for sample_labels in nearest_neighbors_labels:
            frequency = np.zeros(10)
            for label in sample_labels:
                frequency[label] += 1
            result.append(frequency / self.k)
        return np.array(result)
      
    def predict(self, X):        
        nearest_neighbors_labels = self.__get_nearest_neighbors_labels(X)
        return np.array([np.argmax(np.bincount(labels)) for labels in nearest_neighbors_labels])
    
    def __get_nearest_neighbors_labels(self, X):
        distances = np.empty((len(X), len(self.__X)), dtype=np.ndarray)
        for test_sample_ind, test_sample in enumerate(tqdm(X)):
            for train_object_ind, train_object in enumerate(self.__X):
                distances[test_sample_ind, train_object_ind] = (self.distance_func(test_sample, train_object),
                                                                self.__y[train_object_ind])
                
        sorted_nearest_distances = \
            np.array([np.sort(current_distances)[0:self.k] for current_distances in distances])
        
        nearest_neighbors_labels = \
            np.array([np.array([label for _, label in nearest_distances]) for nearest_distances in sorted_nearest_distances])
        
        return nearest_neighbors_labels
        

## 3. Check correctness (compare with exist implementations)

In [9]:
def euclidean_distance(lhs, rhs):
    return np.sqrt(np.sum((lhs - rhs)**2))

## 3.1. Собственная реализация

In [10]:
knn = KNeighborsClassifier(euclidean_distance, k=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

100%|██████████| 100/100 [00:01<00:00, 58.03it/s]


[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  1  6  0  1  0  0  0  0  0]
 [ 0  1  1  8  0  0  0  0  0  1]
 [ 0  0  0  0 11  0  0  0  0  3]
 [ 0  1  0  1  1  4  0  0  0  0]
 [ 1  0  0  0  1  0  8  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  1]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  2  0  9]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.82      1.00      0.90        14
           2       0.86      0.75      0.80         8
           3       0.89      0.73      0.80        11
           4       0.79      0.79      0.79        14
           5       1.00      0.57      0.73         7
           6       1.00      0.80      0.89        10
           7       0.88      0.93      0.90        15
           8       1.00      1.00      1.00         2
           9       0.64      0.82      0.72        11

    accuracy                           0.84       100
   macro avg       

In [11]:
print(knn.predict_proba(X_test)[:7])

[[0.  0.  0.  0.  0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.6 0.2 0.  0.  0.  0.  0.2 0. ]
 [0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.6 0.  0.  0.2 0.  0.2]
 [0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.6]]


## 3.2. Существующая реализация

In [12]:
from sklearn import neighbors

classifier = neighbors.KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)
y_pred_sklearn = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred_sklearn))
print(classification_report(y_test, y_pred_sklearn))

[[ 8  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  1  6  0  1  0  0  0  0  0]
 [ 0  1  1  8  0  0  0  0  0  1]
 [ 0  0  0  0 11  0  0  0  0  3]
 [ 0  1  0  1  1  4  0  0  0  0]
 [ 1  0  0  0  1  0  8  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  1]
 [ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  2  0  9]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.82      1.00      0.90        14
           2       0.86      0.75      0.80         8
           3       0.89      0.73      0.80        11
           4       0.79      0.79      0.79        14
           5       1.00      0.57      0.73         7
           6       1.00      0.80      0.89        10
           7       0.88      0.93      0.90        15
           8       1.00      1.00      1.00         2
           9       0.64      0.82      0.72        11

    accuracy                           0.84       100
   macro avg       

In [13]:
print(classifier.predict_proba(X_test)[:7])

[[0.  0.  0.  0.  0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.6 0.2 0.  0.  0.  0.  0.2 0. ]
 [0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.6 0.  0.  0.2 0.  0.2]
 [0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.4 0.  0.  0.  0.  0.6]]


## 4. Define features and distance functions

In [9]:
from skimage.feature import hog

In [10]:
def get_hog_image(image):
    __image = image
    __image_size = int(np.sqrt(len(__image)))
    __image = __image.reshape(__image_size, __image_size)
    _, __image = hog(__image, visualize=True, multichannel=False)
    __image = __image.flatten()
    return __image

def get_histogram(image):
    hist, _ = np.histogram(image, bins=256)
    return hist

In [11]:
def manhattan_distance(lhs, rhs):
    return np.sum(np.abs(lhs - rhs))

In [12]:
def hog_image_euclidean_distance(lhs, rhs):
    return euclidean_distance(get_hog_image(lhs), get_hog_image(rhs))

In [13]:
def hog_image_manhattan_distance(lhs, rhs):
    return manhattan_distance(get_hog_image(lhs), get_hog_image(rhs))

In [14]:
def hist_euclidean_distance(lhs, rhs):
    return euclidean_distance(get_histogram(lhs), get_histogram(rhs))

In [15]:
def hist_manhattan_distance(lhs, rhs):
    return manhattan_distance(get_histogram(lhs), get_histogram(rhs))

## 5. Find hyper-parameters

In [21]:
K = [3, 5, 7]
dist_functions_and_features = [('euclidean', euclidean_distance),
                               ('manhattan', manhattan_distance),
                               ('hog_image_euclidean_distance', hog_image_euclidean_distance),
                               ('hog_image_manhattan_distance', hog_image_manhattan_distance),
                               ('hist_euclidean_distance', hist_euclidean_distance),
                               ('hist_manhattan_distance', hist_manhattan_distance)]

In [22]:
def find_best(distance_functions, K):
    result_list = []
    for distance_name, distance_func in distance_functions:
        for k in K:
            knn = KNeighborsClassifier(distance_func, k=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            
            result = accuracy_score(y_test, y_pred)
            result_list.append((result, [distance_name, k]))
            
    return {k: v for k, v in sorted(result_list, key=lambda item: item[0], reverse=True)}

In [23]:
grid_result = find_best(dist_functions_and_features, K)
for item in grid_result.items():
    print(item)

(0.92, ['hog_image_manhattan_distance', 7])
(0.91, ['hog_image_manhattan_distance', 3])
(0.9, ['hog_image_euclidean_distance', 5])
(0.86, ['hog_image_euclidean_distance', 3])
(0.84, ['euclidean', 5])
(0.83, ['euclidean', 3])
(0.82, ['manhattan', 3])
(0.81, ['euclidean', 7])
(0.8, ['manhattan', 5])
(0.79, ['manhattan', 7])
(0.37, ['hist_euclidean_distance', 5])
(0.32, ['hist_manhattan_distance', 5])
(0.31, ['hist_manhattan_distance', 7])
(0.26, ['hist_euclidean_distance', 3])
(0.24, ['hist_manhattan_distance', 3])


## 5.1 Full dataset

### 5.1.1 Convert data

In [16]:
for i in tqdm(range(len(X_train))):
    X_train[i] = get_hog_image(X_train[i])

100%|██████████| 60000/60000 [01:16<00:00, 782.52it/s]


In [18]:
for j in tqdm(range(len(X_test))):
    X_test[j] = get_hog_image(X_test[j])

100%|██████████| 10000/10000 [00:12<00:00, 781.40it/s]


### 5.1.2 Train data

In [ ]:
knn = KNeighborsClassifier(manhattan_distance, k=7)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
            
print(accuracy_score(y_test, y_pred))

  4%|▍         | 392/10000 [03:33<1:25:22,  1.88it/s]

## 6. Results
### The best model parameters:
* k: 7
* Distance function: manhattan
* Features: HOG

### The best model test accuracy: 0.92

###  Why such model parameters are the best?

От противного:
* Гистограммы цветов для этой задачи не имеют никаких преимуществ, т.к. размеры и цвет цифр примерно одинаковые для всех сэмплов (изначально пробовал корреляцию и кросс-корреляцию, но там только хуже было), поэтому явного отделения классов не будет.
* Евклидово расстояние и расстояние Манхеттена имеют не плохие результаты, потому что здесь хорошие данные, все цифры плюс минус в центре и одинаковых размеров (нет сдвигов и масштабов), поэтому явное сравнение пикселей дает такие результаты.
* Гистограммы направленных градиентов выигрывают, потому что в некотором смысле данный дескриптор оценивает форму объекта, что положительно сказывается на результатах обучения.